### Import Dependencies

In [1]:
import random
random.seed(0)

In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.transforms import v2

### Constant Variables

In [3]:
DATASET_PATH = './knee-osteoarthritis'

In [4]:
TRAIN_PATH = f'{DATASET_PATH}/train'
VAL_PATH = f'{DATASET_PATH}/val'
TEST_PATH = f'{DATASET_PATH}/test'
AUTO_TEST_PATH = f'{DATASET_PATH}/auto_test'

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


### Dataset 

In [6]:
from src.dataset.augmented_dataset import get_KneeOsteoarthritis_Edges

train_dataset = get_KneeOsteoarthritis_Edges(TRAIN_PATH)
val_dataset = get_KneeOsteoarthritis_Edges(VAL_PATH)

In [7]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

In [8]:
row = train_dataset[1]
normal_ex = row[0]
augmented_ex = row[1]
print(normal_ex.shape, augmented_ex.shape)

torch.Size([3, 256, 256]) torch.Size([1, 224, 224])


### Data Loader

In [9]:
from torch.utils.data import DataLoader
BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

### Model

In [128]:
class IntermediarySpaceModel(nn.Module):
    def __init__(self, num_classes: int = 5, dropout: float = 0.5) -> None:
        super().__init__()
        
        # Size of layer block
        S = 32
        
        # Images
        self.imagesClassifier = nn.Sequential(
            nn.Conv2d(3, S*2, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(S*2, S*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(S*2, S, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(S * 7 * 7, S*2),
        )

        self.edgesClassifier = nn.Sequential(
            nn.Conv2d(1, S*2, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
            nn.Dropout(p=dropout*0.4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.6),
            nn.Conv2d(S*2, S, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout*0.8),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(S * 6 * 6, S*2),
        )
        
        self.outputCombiner = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(S*4, S*3),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(S*3, S),
            nn.Dropout(p=dropout),
            nn.ReLU(inplace=True),
            nn.Linear(S, num_classes),
        )

    def forward(self, images: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        
        # Images
        images = self.imagesClassifier(images)
        
        # Edges
        edges = self.edgesClassifier(edges)
        
        # Combining outputs
        concated = torch.cat((images, edges), 1)
        res = self.outputCombiner(concated)
        
        return res

In [129]:
# from src.models.custom import CustomModel

model = IntermediarySpaceModel(3, 0.5)
model = model.to(device)

In [130]:
# print(sum(p.numel() for p in net.classifier.parameters()) ,sum(p.numel() for p in net.edgesClassifier.parameters()) )
print(sum(p.numel() for p in model.parameters()))

trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
print(sum(p.numel() for p in trainable_parameters))

499683
499683


### Training Loop

#### Setting optimizer

In [131]:
import torch.optim as optim
from src.other import getClassesFrequency

class_weights = getClassesFrequency(train_dataset)
weights_tensor = torch.Tensor(list(class_weights.values())).to(device)

criterion = nn.CrossEntropyLoss(weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [132]:
def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

#### Setting Logger

In [133]:
EXP_NAME = "4/3"

In [134]:
from torch.utils.tensorboard import SummaryWriter

logger = SummaryWriter(log_dir=f"logs/experiments/{EXP_NAME}")

In [135]:
epochCounter = 0

#### Training Loop

In [136]:
from src.validation import validate

def train_many(model, epochs_nr, regularization_type = "L2", lambda_reg=0.01):
    global epochCounter
    
    for epoch in range(0, epochs_nr):  # loop over the dataset multiple times
        epoch_correct = 0
        epoch_samples = 0
        epoch_batches = 0
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):

            # get the inputs; data is a list of [inputs, labels]
            images, edges, labels = data
            images = images.to(device)
            edges = edges.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(images, edges)
            loss = criterion(outputs, labels)
            
            # Apply L1 regularization
            if regularization_type == 'L1':
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss += lambda_reg * l1_norm
                
            # Apply L2 regularization
            elif regularization_type == 'L2':
                l2_norm = sum(p.pow(2).sum() for p in model.parameters())
                loss += lambda_reg * l2_norm
                
            loss.backward()
            
            optimizer.step()
            
            # Changing outputs (logits) to labels
            outputs_clear = outputs.max(1).indices
            
            epoch_correct += (outputs_clear == labels).float().sum()
            epoch_samples += len(outputs)
            epoch_batches +=1
            
            running_loss += loss.item()
        
        tAccuracy = epoch_correct / epoch_samples * 100
        tLoss = running_loss / epoch_batches
        
        # Validation
        vAccuracy, vLoss = validate(model, val_loader, criterion, device)
        
        logger.add_text("REGULARIZATION_TYPE", regularization_type, global_step=epochCounter)
        logger.add_scalar("REGULARIZATION_LAMBDA", lambda_reg, global_step=epochCounter)
        logger.add_scalar("learning_rate", get_lr(optimizer), global_step=epochCounter)
        
        logger.add_scalar("Accuracy/train", tAccuracy, global_step=epochCounter)
        logger.add_scalar("Loss/train", tLoss, global_step=epochCounter)
        logger.add_scalar("Accuracy/validation", vAccuracy, global_step=epochCounter)
        logger.add_scalar("Loss/validation", vLoss, global_step=epochCounter)
        
        print(f'Epoch {epochCounter}: Training: accuracy: {tAccuracy:.3f}%, loss: {tLoss:.3f}; Validation: accuracy: {vAccuracy:.3f}%, loss: {vLoss:.3f}')
        
        epochCounter += 1
        
    print('Finished Training')

### Training Model

In [137]:
train_many(model, 50, "L2", 0.001)
# train_many(model, 50)

Epoch 0: Training: accuracy: 53.617%, loss: 1.045; Validation: accuracy: 55.327%, loss: 0.712
Epoch 1: Training: accuracy: 57.148%, loss: 0.805; Validation: accuracy: 56.901%, loss: 0.668
Epoch 2: Training: accuracy: 57.459%, loss: 0.777; Validation: accuracy: 56.901%, loss: 0.674
Epoch 3: Training: accuracy: 57.373%, loss: 0.769; Validation: accuracy: 57.022%, loss: 0.665
Epoch 4: Training: accuracy: 57.477%, loss: 0.763; Validation: accuracy: 57.022%, loss: 0.678
Epoch 5: Training: accuracy: 57.442%, loss: 0.754; Validation: accuracy: 57.022%, loss: 0.660
Epoch 6: Training: accuracy: 57.407%, loss: 0.750; Validation: accuracy: 57.022%, loss: 0.656
Epoch 7: Training: accuracy: 57.425%, loss: 0.741; Validation: accuracy: 56.901%, loss: 0.663
Epoch 8: Training: accuracy: 57.425%, loss: 0.734; Validation: accuracy: 57.022%, loss: 0.668
Epoch 9: Training: accuracy: 57.407%, loss: 0.727; Validation: accuracy: 57.022%, loss: 0.656
Epoch 10: Training: accuracy: 57.425%, loss: 0.723; Validati